# Homework 1

#### EE-556 Mathematics of Data - Fall 2024


In this homework, we consider a multiclass classification task modeled by multinomial (softmax) logistic regression. Your goal will be to analyze the estimator and its properties (convexity, existence/uniqueness), and to derive gradients/Hessians and smoothness bounds. The first part consists of theoretical questions only.


<div class="alert alert-info">
  ℹ️ <strong>Information on group based work:</strong>
</div>

- You are to deliver only 1 notebook per group.
- Asking assistance beyond your group is ok, but answers should be individual to the group.
- In the event that there was <span style="color: red;">disproportional work done</span> by different group members, let the TAs know.
- Only one member of the group is allowed to use AI. We will require sharing the conversation history with the AI in the form of a public link. If you use multiple conversations across the same or multiple tools please share all of them. Name the person in your group who is allowed to use AI. We encourage you to use the AI to help you understand the material, but we ask you to write the code and theory solutions by yourself.

<div style="border: 1px solid #f00; background-color: #fdd; padding: 10px; border-radius: 5px;">
  ⚠️ Do not forget: Write who are the people in your group as well as their respective SCIPER numbers
</div>


Person 1 **Luis Falke**: || Person 1 **414954**:


Person 2 **Name**: || Person 2 **SCIPER**:


Person 3 **Name**: || Person 3 **SCIPER**:


<div style="border: 1px solid #0a0; background-color: #dfd; padding: 10px; border-radius: 5px;">
  📓 Feedback on AI use: Please use the following cell to provide feedback on the AI use in this notebook.
  
  For example, how useful were the tools to you? Which tools did you use? Did you feel like they helped you understand the material better?
</div

## 1. Multiclass Softmax Logistic Regression - 15 Points


We now model multiclass classification with classes $c \in \{1,\dots,C\}$. For each sample $(\mathbf{a}_i, b_i)$ with $\mathbf{a}_i \in \mathbb{R}^p$ and $b_i \in \{1,\dots,C\}$, let $\mathbf{X} = [\mathbf{x}_1,\dots,\mathbf{x}_C] \in \mathbb{R}^{p\times C}$ be the class weight matrix. The softmax model defines

$$
\mathbb{P}(b_i = c \mid \mathbf{a}_i) = \frac{\exp(\mathbf{a}_i^\top \mathbf{x}_c)}{\sum_{k=1}^C \exp(\mathbf{a}_i^\top \mathbf{x}_k)}.
$$

Assume i.i.d. samples $\{(\mathbf{a}_i,b_i)\}_{i=1}^n$. Our goal is to estimate $\mathbf{X}$ by maximum likelihood (and later with an $\ell_2$ regularizer).


__(a)__ (1 point) Show that the negative log-likelihood $f$ can be written as:

$$
\begin{aligned}
 f(\mathbf{X})
 &= - \log \mathbb{P}(b_1,\dots,b_n\mid \mathbf{a}_1,\dots,\mathbf{a}_n)\\
 &= \sum_{i=1}^n \left[ -\mathbf{a}_i^\top \mathbf{x}_{b_i} + \log \sum_{k=1}^C \exp(\mathbf{a}_i^\top \mathbf{x}_k) \right].
\end{aligned}
$$



### Solution

For one sample $(\mathbf{a}_i, b_i)$, the log-probability of the observed class $b_i$ is

$$
\log \mathbb{P}(b_i \mid \mathbf{a}_i)
= \log \frac{\exp(\mathbf{a}_i^\top \mathbf{x}_{b_i})}{\sum_{k=1}^C \exp(\mathbf{a}_i^\top \mathbf{x}_k)}
= \mathbf{a}_i^\top \mathbf{x}_{b_i} - \log\left(\sum_{k=1}^C \exp(\mathbf{a}_i^\top \mathbf{x}_k)\right).
$$

Assuming i.i.d. samples, the likelihood is the product

$$
\mathbb{P}(b_1,\dots,b_n \mid \mathbf{a}_1,\dots,\mathbf{a}_n, \mathbf{X})
= \prod_{i=1}^n \mathbb{P}(b_i \mid \mathbf{a}_i).
$$

Taking the negative log gives the **negative log-likelihood**

$$
f(\mathbf{X}) = -\log \mathbb{P}(b_1,\dots,b_n \mid \mathbf{a}_1,\dots,\mathbf{a}_n, \mathbf{X})
= -\sum_{i=1}^n \log \mathbb{P}(b_i \mid \mathbf{a}_i),
$$

which simplifies to

$$
f(\mathbf{X})
= \sum_{i=1}^n \left[
 -\,\mathbf{a}_i^\top \mathbf{x}_{b_i}
 + \log\left(\sum_{k=1}^C \exp(\mathbf{a}_i^\top \mathbf{x}_k)\right)
\right].
$$


This is exactly the desired form of the negative log-likelihood for multiclass softmax logistic regression.


__(b)__ (2 points) Show that $\mathbf{u} \mapsto \log\!\left(\sum_{k=1}^C e^{u_k}\right)$ is convex on $\mathbb{R}^C$. Then, show that $f(\mathbf{X})$ is convex.


Hint: use Jensen's inequality.

### Solution
#### Step 1: $\mathrm{lse}(\mathbf{u}) := \log\!\left(\sum_{k=1}^C e^{u_k}\right)$ is convex on $\mathbb{R}^C$.

Let $\Delta_C := \{\boldsymbol{\theta}\in\mathbb{R}^C_{\ge 0} : \sum_{k=1}^C \theta_k = 1\}$ be the probability simplex.  
Because $\log$ is **concave**, Jensen’s inequality says that for any positive numbers $t_k>0$ and any $\boldsymbol{\theta}\in\Delta_C$,
$$
\log\!\Big(\sum_{k=1}^C \theta_k t_k\Big)\;\ge\;\sum_{k=1}^C \theta_k \log t_k,
$$
with equality iff $t_k$ are all equal for every index with $\theta_k>0$.

Apply this with $t_k=\dfrac{e^{u_k}}{\theta_k}$ (if some $\theta_k=0$, read the expression by continuity; the inequality still holds). Then
$$
\begin{aligned}
\mathrm{lse}(\mathbf{u})
&= \log\!\left(\sum_{k=1}^C e^{u_k}\right)
= \log\!\left(\sum_{k=1}^C \theta_k \frac{e^{u_k}}{\theta_k}\right)
\\
&\ge \sum_{k=1}^C \theta_k \log\!\left(\frac{e^{u_k}}{\theta_k}\right)
= \sum_{k=1}^C \theta_k u_k \;-\; \sum_{k=1}^C \theta_k \log \theta_k.
\end{aligned}
\tag{1}
$$
Since $\boldsymbol{\theta}$ was arbitrary, we obtain the variational form
$$
\mathrm{lse}(\mathbf{u})
\;\ge\;
\sup_{\boldsymbol{\theta}\in\Delta_C}
\Big\{\,\boldsymbol{\theta}^\top \mathbf{u} - \sum_{k=1}^C \theta_k \log \theta_k\,\Big\}.
\tag{2}
$$

**Where does equality hold?**  
Equality in Jensen for a concave function holds iff all inputs to $\log$ are equal on the support of $\boldsymbol{\theta}$.  
Here those inputs are $t_k=\dfrac{e^{u_k}}{\theta_k}$. Thus equality requires a constant $Z>0$ such that
$$
\frac{e^{u_k}}{\theta_k} = Z \quad \text{for all } k \text{ with } \theta_k>0,
$$
equivalently
$$
\theta_k = \frac{e^{u_k}}{Z}.
$$
Enforcing $\sum_k \theta_k=1$ gives $Z=\sum_{j=1}^C e^{u_j}$. Therefore the maximizer is
$$
\boxed{\,\theta_k^*(\mathbf{u})=\dfrac{e^{u_k}}{\sum_{j=1}^C e^{u_j}}\,}.
$$
Plugging $\boldsymbol{\theta}^*$ into the right-hand side of (1),
$$
\sum_{k} \theta_k^* u_k - \sum_{k} \theta_k^* \log \theta_k^*
= \sum_k \theta_k^* u_k - \sum_k \theta_k^*(u_k - \log Z)
= \log Z
= \log\!\left(\sum_{j=1}^C e^{u_j}\right)
= \mathrm{lse}(\mathbf{u}),
$$
so the inequality is tight and we have the exact identity
$$
\boxed{\;\mathrm{lse}(\mathbf{u}) = \sup_{\boldsymbol{\theta}\in\Delta_C}
\Big\{\,\boldsymbol{\theta}^\top \mathbf{u} - \sum_{k=1}^C \theta_k \log \theta_k\,\Big\}. \;}
\tag{3}
$$

For each fixed $\boldsymbol{\theta}$, the map $\mathbf{u}\mapsto \boldsymbol{\theta}^\top \mathbf{u} - \sum_k \theta_k \log \theta_k$ is **affine** in $\mathbf{u}$. A supremum of affine functions is convex, hence $\mathrm{lse}$ is convex on $\mathbb{R}^C$.

---

#### Step 2: Convexity of $f(\mathbf{X})$.

From part (a),
$$
f(\mathbf{X})
= \sum_{i=1}^n \left[
-\,\mathbf{a}_i^\top \mathbf{x}_{b_i}
+ \log\!\left(\sum_{k=1}^C e^{\mathbf{a}_i^\top \mathbf{x}_k}\right)
\right]
= \sum_{i=1}^n \left[
-\,\mathbf{a}_i^\top \mathbf{x}_{b_i}
+ \mathrm{lse}\!\big(\mathbf{u}_i(\mathbf{X})\big)
\right],
$$
where $\mathbf{u}_i(\mathbf{X}) := \big(\mathbf{a}_i^\top \mathbf{x}_1,\dots,\mathbf{a}_i^\top \mathbf{x}_C\big)\in\mathbb{R}^C$.

- The map $\mathbf{X}\mapsto \mathbf{u}_i(\mathbf{X})$ is **affine** in $\mathbf{X}$.
- Since $\mathrm{lse}$ is convex, the composition $\mathbf{X}\mapsto \mathrm{lse}\!\big(\mathbf{u}_i(\mathbf{X})\big)$ is convex (convex $\circ$ affine).
- Sums of convex functions are convex, and adding the linear term $-\,\mathbf{a}_i^\top \mathbf{x}_{b_i}$ preserves convexity.

Therefore $f(\mathbf{X})$ is convex in $\mathbf{X}$.


You have just established that the negative log-likelihood is a convex function. So in principle, any local minimum of the maximum likelihood estimator
$$
\mathbf{X}^\star_{ML} = \arg\min_{\mathbf{X} \in \mathbb{R}^{p\times C}} f(\mathbf{X})
$$

is a global minimum. But does the minimum always exist? We will ponder this question in the following three points.


__(c)__ (1 point) Explain the difference between infima and minima. Give an example of a convex function on $\mathbb{R}$ that does not attain its infimum.


### Solution

- **Infimum:** The infimum of $f$ over a set $S$ is the greatest lower bound of $\{f(x): x\in S\}$.  
  In symbols,
  $$
  \inf_{x\in S} f(x) \;=\; \sup\{\alpha\in\mathbb{R} : \alpha \le f(x)\ \text{for all } x\in S\}.
  $$
  It need not be achieved by any $x\in S$.

- **Minimum:** A minimum is a point $x^\star\in S$ such that $f(x^\star)\le f(x)$ for all $x\in S$; equivalently,
  $$
  f(x^\star) \;=\; \inf_{x\in S} f(x),
  $$
  i.e., the infimum is **attained** at $x^\star$.

**Example (convex on $\mathbb{R}$ with no minimum):**  
Let $f(x)=e^x$. Then $f''(x)=e^x>0$, so $f$ is convex on $\mathbb{R}$.  
We have
$$
\inf_{x\in\mathbb{R}} f(x) \;=\; 0 \quad \text{(since $e^x\to 0$ as $x\to -\infty$),}
$$
but $e^x>0$ for every $x$, so there is **no** $x^\star$ with $f(x^\star)=0$.  
Thus the infimum is **not attained**-$f$ has no minimizer on $\mathbb{R}$.


__(d)__ (1 point) Assume there exists $\mathbf{X}_0 \in \mathbb{R}^{p\times C}$ such that for all $i$,
$$
\mathbf{a}_i^\top \mathbf{x}_{0, b_i} - \max_{k \neq b_i} \mathbf{a}_i^\top \mathbf{x}_{0,k} > 0.
$$
This is called one-versus-all complete separation in multiclass settings. Give a geometric interpretation (e.g., for $p=2$) and explain why the name is appropriate.


### Solution

Assume there exists $\mathbf{X}_0 \in \mathbb{R}^{p\times C}$ such that for every sample $i$,
$$
\mathbf{a}_i^\top \mathbf{x}_{0,b_i} \;-\; \max_{k\ne b_i}\,\mathbf{a}_i^\top \mathbf{x}_{0,k} \;>\; 0.
\tag{*}
$$
Write the columns of $\mathbf{X}_0$ as $\mathbf{x}_{0,1},\dots,\mathbf{x}_{0,C}$ and define the **scores**
$$
s_c(\mathbf{a}) := \mathbf{a}^\top \mathbf{x}_{0,c}, \qquad c=1,\dots,C.
$$
The class predicted by the linear (softmax) model is $\arg\max_c s_c(\mathbf{a})$.

---

#### Step 1: Pairwise decision boundaries
For any two classes $c$ and $k$, the boundary where they tie is
$$
\{\mathbf{a} : s_c(\mathbf{a}) = s_k(\mathbf{a})\}
\;\Longleftrightarrow\;
\{\mathbf{a} : \mathbf{a}^\top(\mathbf{x}_{0,c}-\mathbf{x}_{0,k})=0\}.
$$
This set is a hyperplane **through the origin** with normal vector $\mathbf{x}_{0,c}-\mathbf{x}_{0,k}$.  
(It passes through the origin because there is no intercept; with an intercept one can augment $\mathbf{a}$ by a $1$ to keep the same form.)

Define the half-spaces
$$
H_{c\succ k} := \{\mathbf{a} : \mathbf{a}^\top(\mathbf{x}_{0,c}-\mathbf{x}_{0,k}) \ge 0\}
\quad\text{and}\quad
H_{k\succ c} := \{\mathbf{a} : \mathbf{a}^\top(\mathbf{x}_{0,c}-\mathbf{x}_{0,k}) \le 0\}.
$$
Points in $H_{c\succ k}$ are those where class $c$’s score is at least as large as class $k$’s.

---

#### Step 2: From $(*)$ to per-class conic regions
The condition $(*)$ is equivalent to a **family of strict pairwise inequalities**:

- Since $\max_{k\ne b_i} s_k(\mathbf{a}_i)$ is the largest competitor score,
  $$
  s_{b_i}(\mathbf{a}_i) - \max_{k\ne b_i} s_k(\mathbf{a}_i) > 0
  \;\Longrightarrow\;
  s_{b_i}(\mathbf{a}_i) - s_k(\mathbf{a}_i) > 0 \quad \forall\, k\ne b_i,
  $$
  i.e.
  $$
  \mathbf{a}_i^\top(\mathbf{x}_{0,b_i}-\mathbf{x}_{0,k}) > 0 \quad \forall\, k\ne b_i.
  \tag{1}
  $$

- Conversely, if (1) holds for all $k\ne b_i$, then $s_{b_i}(\mathbf{a}_i)$ is strictly larger than **every** $s_k(\mathbf{a}_i)$, hence larger than their maximum; thus $(*)$ holds.  
  Therefore, $(*) \Longleftrightarrow$ (1).

For each class $c$, define the **one-versus-all region**
$$
\mathcal{R}_c := \bigcap_{k\ne c} H_{c\succ k}
= \left\{\mathbf{a} : \mathbf{a}^\top(\mathbf{x}_{0,c}-\mathbf{x}_{0,k}) \ge 0 \ \ \forall\,k\ne c\right\}.
$$
This is an intersection of $C-1$ half-spaces, hence a closed **convex polyhedral cone** (a wedge when $p=2$).  
Its **interior** is
$$
\operatorname{int}(\mathcal{R}_c) = \left\{\mathbf{a} : \mathbf{a}^\top(\mathbf{x}_{0,c}-\mathbf{x}_{0,k}) > 0 \ \ \forall\,k\ne c\right\}.
$$

By (1), every data point of class $b_i$ lies in $\operatorname{int}(\mathcal{R}_{b_i})$ with a **strict margin** to all the pairwise hyperplanes.

---

#### Step 3: Geometric interpretation (take $p=2$ for a picture)
When $p=2$, each boundary $\{\mathbf{a} : \mathbf{a}^\top(\mathbf{x}_{0,c}-\mathbf{x}_{0,k}) = 0\}$ is a **line through the origin** with normal $\mathbf{x}_{0,c}-\mathbf{x}_{0,k}$.  
These lines partition the plane into at most $C$ wedge-shaped (conic) regions. The region $\mathcal{R}_c$ is the wedge where $s_c(\mathbf{a})$ dominates all other scores.  
Condition $(*)$ says each point of class $c$ lies **strictly inside** that wedge, not on any boundary.  
Hence different classes occupy **disjoint regions** separated by these linear boundaries, with a positive margin.

---

#### Step 4: Why the name “one-versus-all complete separation”?
- **One-versus-all:** For each class $c$, the requirement is
  $$
  s_c(\mathbf{a}) > s_k(\mathbf{a}) \quad \text{for all } k\ne c,
  $$
  i.e., the score of $c$ beats the scores of **all** other classes at every point of class $c$.
- **Complete:** The inequalities are **strict** (“$>$”), so there are no ties; every point is strictly on the correct side of every pairwise hyperplane.
- **Separation:** Because each class’s points lie in the interior of $\mathcal{R}_c$, the class regions are disjoint and linearly separated by the hyperplanes
  $$
  \{\mathbf{a} : \mathbf{a}^\top(\mathbf{x}_{0,c}-\mathbf{x}_{0,k}) = 0\}, \qquad c\ne k.
  $$

Thus the assumption $(*)$ exactly states **one-versus-all complete separation** in the multiclass linear (softmax) setting.


From this, you should see that it is likely that some datasets satisfy the complete separation assumption. Unfortunately, as you will show next, this can become an obstacle.


__(e)__ (1 point) In a one-versus-all complete separation setting (as in (d)), prove that $f$ does not attain its minimum. Hint: consider $f(\alpha \mathbf{X}_0)$ as $\alpha \to +\infty$ and compare it to $f(\mathbf{X}_0)$.


### Solution

Let $\mathbf{X}_0=[\mathbf{x}_{0,1},\ldots,\mathbf{x}_{0,C}]$ satisfy the separation condition from (d):
$$
\mathbf{a}_i^\top \mathbf{x}_{0,b_i} \;-\; \max_{k\ne b_i} \mathbf{a}_i^\top \mathbf{x}_{0,k} \;>\; 0
\qquad\forall i.
$$

Define the per-example **pairwise margins**
$$
\Delta_{ik} \;:=\; \mathbf{a}_i^\top \mathbf{x}_{0,b_i} - \mathbf{a}_i^\top \mathbf{x}_{0,k} \;>\; 0,
\qquad
\Delta_i \;:=\; \min_{k\ne b_i}\Delta_{ik} \;>\; 0,
$$
and let $\gamma := \min_i \Delta_i > 0$ (the minimum over finitely many strictly positive numbers).

Consider the ray $\mathbf{X}(\alpha) := \alpha \mathbf{X}_0$ with $\alpha>0$.  
Using the expression from (a),
$$
\begin{aligned}
f(\mathbf{X}(\alpha))
&= \sum_{i=1}^n \left[-\,\alpha\,\mathbf{a}_i^\top \mathbf{x}_{0,b_i}
+ \log\!\left(\sum_{k=1}^C e^{\alpha\,\mathbf{a}_i^\top \mathbf{x}_{0,k}}\right)\right] \\
&= \sum_{i=1}^n \log\!\left(\sum_{k=1}^C e^{\alpha(\mathbf{a}_i^\top \mathbf{x}_{0,k}-\mathbf{a}_i^\top \mathbf{x}_{0,b_i})}\right) \\
&= \sum_{i=1}^n \log\!\left( 1 + \sum_{k\ne b_i} e^{-\alpha\,\Delta_{ik}} \right).
\end{aligned}
$$

**Upper bound and limit.**  
Because $\Delta_{ik}\ge \Delta_i \ge \gamma$,
$$
f(\mathbf{X}(\alpha))
\;\le\; \sum_{i=1}^n \log\!\left(1 + (C-1)e^{-\alpha \Delta_i}\right)
\;\le\; n\,\log\!\left(1 + (C-1)e^{-\alpha \gamma}\right)
\;\xrightarrow[\alpha\to\infty]{}\; 0.
$$

**Lower bound (nonnegativity).**  
For any $\mathbf{X}$,
$$
-\,\mathbf{a}_i^\top \mathbf{x}_{b_i} + \log\!\left(\sum_k e^{\mathbf{a}_i^\top \mathbf{x}_k}\right)
\;\ge\; -\,\mathbf{a}_i^\top \mathbf{x}_{b_i} + \log\!\left(e^{\mathbf{a}_i^\top \mathbf{x}_{b_i}}\right)
\;=\; 0,
$$
so $f(\mathbf{X})\ge 0$ for all $\mathbf{X}$. Hence $\inf_{\mathbf{X}} f(\mathbf{X}) = 0$.

**Non-attainment.**  
If $f$ attained its minimum at some finite $\mathbf{X}^\star$, we would have $f(\mathbf{X}^\star)=0$.  
Since each summand is nonnegative, each must equal $0$, i.e. for every $i$,
$$
0 \;=\; \log\!\left(\sum_{k=1}^C e^{\mathbf{a}_i^\top \mathbf{x}_k^\star}\right)
- \mathbf{a}_i^\top \mathbf{x}_{b_i}^\star
\quad\Longrightarrow\quad
\sum_{k\ne b_i} e^{\mathbf{a}_i^\top \mathbf{x}_k^\star} \;=\; 0,
$$
which is impossible because every term $e^{\mathbf{a}_i^\top \mathbf{x}_k^\star}$ is strictly positive.  
Therefore no finite $\mathbf{X}$ achieves $f(\mathbf{X})=0$, and the minimum is **not attained**.

**Conclusion.**  
Under complete separation, the negative log-likelihood has infimum $0$ but **no minimizer**.  
Along the ray $\mathbf{X}(\alpha)=\alpha\mathbf{X}_0$ the objective decreases to $0$ as $\alpha\to\infty$, reflecting that the model pushes class probabilities to $1$ for the observed labels while parameters diverge.


We resolve this issue by adding a regularizer. Consider the regularized function

$$
 f_\mu(\mathbf{X}) = f(\mathbf{X}) + \frac{\mu}{2} \|\mathbf{X}\|_F^2, \quad \mu > 0.
$$

__(f)__ (1 point) Show that the gradient with respect to $\mathbf{X}$ of $f_\mu$ can be expressed as
$$
 \nabla_{\mathbf{X}} f_\mu(\mathbf{X}) = \sum_{i=1}^n \mathbf{a}_i \big( \mathbf{p}_i - \mathbf{e}_{b_i} \big)^\top + \mu \mathbf{X},\tag{1}
$$
where $\mathbf{e}_{b_i} \in \mathbb{R}^C$ is the [one-hot vector](https://en.wikipedia.org/wiki/One-hot) for class $b_i$, $\mathbf{p}_i \in \mathbb{R}^C$ has entries $p_{i,c} = \mathbb{P}(b_i=c\mid \mathbf{a}_i)$ under the softmax model, and $(\mathbf{p}_i - \mathbf{e}_{b_i})\mathbf{a}_i^\top$ denotes the outer product.



### Solution

We regularize the multiclass softmax NLL with an $\ell_2$ term:
$$
f_\mu(\mathbf{X}) \;=\; f(\mathbf{X}) \;+\; \frac{\mu}{2}\,\|\mathbf{X}\|_F^2,
\qquad \mu>0,
$$
where the unregularized loss is
$$
f(\mathbf{X}) \;=\; \sum_{i=1}^n \Bigg[
-\,\mathbf{a}_i^\top \mathbf{x}_{b_i}
+ \log\!\left(\sum_{k=1}^C e^{\mathbf{a}_i^\top \mathbf{x}_k}\right)
\Bigg],
$$
and $\mathbf{x}_c$ is column $c$ of $\mathbf{X}\in\mathbb{R}^{p\times C}$.

---

#### 1) Re-parameterize one example
For a single example $i$, define
$$
\mathbf{z}_i \;=\; \mathbf{X}^\top \mathbf{a}_i \in \mathbb{R}^C,
\qquad
\mathbf{p}_i \;=\; \mathrm{softmax}(\mathbf{z}_i) \in \mathbb{R}^C,
$$
and let $\mathbf{e}_{b_i}\in\mathbb{R}^C$ be the one-hot vector of the true class $b_i$.
The $i$-th contribution to the loss can be written as
$$
\phi_i(\mathbf{X})
\;=\;
-\,\mathbf{e}_{b_i}^\top \mathbf{z}_i \;+\; \mathrm{lse}(\mathbf{z}_i),
\qquad
\text{where}\quad
\mathrm{lse}(\mathbf{z})=\log\!\sum_{k=1}^C e^{z_k}.
$$

---

#### 2) Gradient with respect to $\mathbf{z}_i$
A standard fact (or by componentwise differentiation) is
$$
\nabla_{\mathbf{z}}\,\mathrm{lse}(\mathbf{z}) \;=\; \mathrm{softmax}(\mathbf{z}).
$$
Therefore
$$
\nabla_{\mathbf{z}_i}\,\phi_i(\mathbf{X})
\;=\;
-\,\mathbf{e}_{b_i} + \mathrm{softmax}(\mathbf{z}_i)
\;=\;
\mathbf{p}_i - \mathbf{e}_{b_i}
\;\in\; \mathbb{R}^C.
$$

---

#### 3) Chain rule to $\mathbf{X}$
Since $\mathbf{z}_i=\mathbf{X}^\top \mathbf{a}_i$, a variation $d\mathbf{X}$ yields $d\mathbf{z}_i=(d\mathbf{X})^\top \mathbf{a}_i$.
Thus, in matrix form,
$$
\nabla_{\mathbf{X}}\,\phi_i(\mathbf{X})
\;=\;
\mathbf{a}_i\,(\mathbf{p}_i - \mathbf{e}_{b_i})^\top,
$$
which is an outer product in $\mathbb{R}^{p\times C}$. Summing over $i$ gives
$$
\nabla_{\mathbf{X}}\,f(\mathbf{X})
\;=\;
\sum_{i=1}^n \mathbf{a}_i\,(\mathbf{p}_i - \mathbf{e}_{b_i})^\top.
$$

*(Equivalently, if $\mathbf{A}=[\mathbf{a}_1,\ldots,\mathbf{a}_n]\in\mathbb{R}^{p\times n}$ and
$\mathbf{P},\mathbf{Y}\in\mathbb{R}^{n\times C}$ stack $\mathbf{p}_i^\top$ and $\mathbf{e}_{b_i}^\top$ as rows,
then $\nabla_{\mathbf{X}} f(\mathbf{X}) = \mathbf{A}(\mathbf{P}-\mathbf{Y})$.)*

---

#### 4) Add the $\ell_2$ regularizer
Using $\dfrac{d}{d\mathbf{X}}\,\tfrac{1}{2}\|\mathbf{X}\|_F^2=\mathbf{X}$,
$$
\nabla_{\mathbf{X}}\!\left(\frac{\mu}{2}\|\mathbf{X}\|_F^2\right) \;=\; \mu\,\mathbf{X}.
$$

---

### Final result
$$
\boxed{\displaystyle
\nabla_{\mathbf{X}} f_\mu(\mathbf{X})
\;=\;
\sum_{i=1}^n \mathbf{a}_i\,(\mathbf{p}_i - \mathbf{e}_{b_i})^\top
\;+\; \mu\,\mathbf{X}}
$$
where $p_{i,c} = \mathbb{P}(b_i=c \mid \mathbf{a}_i)$ under the softmax model, and $(\mathbf{p}_i - \mathbf{e}_{b_i})\,\mathbf{a}_i^\top$ denotes the outer product.


__(g)__ (1 point) Show that the Hessian of $f_\mu$ can be written as
$$
 \nabla^2 f_\mu(\mathbf{X}) = \sum_{i=1}^n (\mathbf{a}_i\mathbf{a}_i^\top) \otimes \big( \operatorname{Diag}(\mathbf{p}_i) - \mathbf{p}_i\mathbf{p}_i^\top \big) + \mu \mathbf{I},\tag{2}
$$
where $\otimes$ is the Kronecker product, and $\operatorname{Diag}(\mathbf{p}_i) - \mathbf{p}_i\mathbf{p}_i^\top$ is the softmax Jacobian, which is positive semidefinite.


### Solution

We differentiate the gradient from (f):
$$
\nabla_{\mathbf{X}} f_\mu(\mathbf{X})
=\sum_{i=1}^n \mathbf{a}_i(\mathbf{p}_i-\mathbf{e}_{b_i})^\top + \mu \mathbf{X},
\qquad
\mathbf{p}_i=\mathrm{softmax}(\mathbf{X}^\top\mathbf{a}_i).
$$
Only the term $\sum_i \mathbf{a}_i\mathbf{p}_i^\top$ depends on $\mathbf{X}$.

Let $ \mathbf{z}_i=\mathbf{X}^\top\mathbf{a}_i\in\mathbb{R}^C $ and
$ J_i:=\nabla_{\mathbf{z}_i}\mathbf{p}_i=\mathrm{Diag}(\mathbf{p}_i)-\mathbf{p}_i\mathbf{p}_i^\top\in\mathbb{R}^{C\times C} $
(the softmax Jacobian).

---

#### Differential $\to$ linear-operator form
Take differentials:
$$
d\mathbf{z}_i=(d\mathbf{X})^\top \mathbf{a}_i,
\qquad
d\mathbf{p}_i=J_i\,d\mathbf{z}_i=J_i (d\mathbf{X})^\top \mathbf{a}_i.
$$
Therefore
$$
d\!\big(\mathbf{a}_i\mathbf{p}_i^\top\big)
=
\mathbf{a}_i (d\mathbf{p}_i)^\top
=
\mathbf{a}_i\big((d\mathbf{z}_i)^\top J_i^\top\big)
=
\mathbf{a}_i(\mathbf{a}_i^\top d\mathbf{X}) J_i
=
(\mathbf{a}_i\mathbf{a}_i^\top)\, d\mathbf{X}\, J_i .
$$
Summing over $i$ and adding the regularizer,
$$
d\!\big(\nabla_{\mathbf{X}} f_\mu(\mathbf{X})\big)
=\sum_{i=1}^n (\mathbf{a}_i\mathbf{a}_i^\top)\, d\mathbf{X}\, J_i + \mu\, d\mathbf{X}.
$$

---

#### Vectorization (Kronecker form)
Using $\mathrm{vec}(A D B)=(B^\top\!\otimes A)\,\mathrm{vec}(D)$ and $J_i^\top=J_i$,
$$
\mathrm{vec}\!\Big(d\!\big(\nabla_{\mathbf{X}} f_\mu(\mathbf{X})\big)\Big)
=
\Bigg[\sum_{i=1}^n \big(J_i \otimes (\mathbf{a}_i\mathbf{a}_i^\top)\big) + \mu I\Bigg]\mathrm{vec}(d\mathbf{X})
=
\Bigg[\sum_{i=1}^n (\mathbf{a}_i\mathbf{a}_i^\top) \otimes J_i + \mu I\Bigg]\mathrm{vec}(d\mathbf{X}).
$$

Hence the Hessian with respect to $\mathbf{X}$ (viewed via $\mathrm{vec}$) is
$$
\boxed{\;
\nabla^2 f_\mu(\mathbf{X})
=\sum_{i=1}^n (\mathbf{a}_i\mathbf{a}_i^\top)\otimes\big(\mathrm{Diag}(\mathbf{p}_i)-\mathbf{p}_i\mathbf{p}_i^\top\big) + \mu I \; }.
$$

---

### Softmax Jacobian is PSD
For any $v\in\mathbb{R}^C$,
$$
v^\top\!\big(\mathrm{Diag}(\mathbf{p}_i)-\mathbf{p}_i\mathbf{p}_i^\top\big)v
=\sum_{c} p_{i,c} v_c^2 - \Big(\sum_{c} p_{i,c} v_c\Big)^2
=\mathrm{Var}_{c\sim \mathbf{p}_i}[v_c]\;\ge 0.
$$
Thus each summand is positive semidefinite; adding $\mu I$ with $\mu>0$ makes the Hessian positive definite, confirming strong convexity of $f_\mu$.


__(h)__ (1 point) Show that $f_\mu$ is $\mu$-strongly convex.


### Solution

From (g), for any $\mathbf{X}$,
$$
\nabla^2 f_\mu(\mathbf{X})
\;=\;
\sum_{i=1}^n (\mathbf{a}_i\mathbf{a}_i^\top)\;\otimes\;
\big(\mathrm{Diag}(\mathbf{p}_i)-\mathbf{p}_i\mathbf{p}_i^\top\big)
\;+\; \mu I .
$$

Each matrix $\mathrm{Diag}(\mathbf{p}_i)-\mathbf{p}_i\mathbf{p}_i^\top$ is positive semidefinite (it is the softmax Jacobian).  
Since $(\mathbf{a}_i\mathbf{a}_i^\top)\succeq 0$ as well and $A\succeq 0$, $B\succeq 0 \Rightarrow A\otimes B\succeq 0$, the Kronecker term
$$
\sum_{i=1}^n (\mathbf{a}_i\mathbf{a}_i^\top)\otimes\big(\mathrm{Diag}(\mathbf{p}_i)-\mathbf{p}_i\mathbf{p}_i^\top\big)
$$
is positive semidefinite. Therefore, for any vector $v$,
$$
v^\top \nabla^2 f_\mu(\mathbf{X}) v
\;\ge\;
v^\top (\mu I) v
\;=\; \mu \|v\|_2^2 .
$$
Equivalently, $\nabla^2 f_\mu(\mathbf{X}) \succeq \mu I$ for all $\mathbf{X}$.

A twice-differentiable function whose Hessian is uniformly bounded below by $\mu I$ is $\mu$-strongly convex (with respect to the Frobenius norm for matrix arguments). Hence, for all $\mathbf{X},\mathbf{Y}$,
$$
f_\mu(\mathbf{Y})
\;\ge\;
f_\mu(\mathbf{X})
\;+\;
\langle \nabla f_\mu(\mathbf{X}),\, \mathbf{Y}-\mathbf{X} \rangle
\;+\;
\frac{\mu}{2}\,\|\mathbf{Y}-\mathbf{X}\|_F^2 .
$$

Therefore, $f_\mu$ is $\mu$-strongly convex.


__(i)__ (1 point) Is it possible for a strongly convex function to not attain its minimum? Justify your reasoning (you may assume the domain is $\mathbb{R}^{p\times C}$).


### Solution

**Answer:** No. On $\mathbb{R}^{p\times C}$ a $\mu$-strongly convex function always **attains** (and has a **unique**) minimizer.

**Why.** By $\mu$-strong convexity, for any fixed $Y$ and all $X$,
$$
f(X)\;\ge\; f(Y)+\langle g_Y,\,X-Y\rangle+\frac{\mu}{2}\|X-Y\|_F^2
\qquad (g_Y\in\partial f(Y)).
$$
Take $Y=0$ and some $g_0\in\partial f(0)$. Then
$$
f(X)\;\ge\; f(0)+\langle g_0, X\rangle+\frac{\mu}{2}\|X\|_F^2
\;\ge\; \frac{\mu}{2}\|X\|_F^2-\|g_0\|_F\,\|X\|_F + f(0).
$$
The right-hand side $\to +\infty$ as $\|X\|_F\to\infty$; thus **$f$ is coercive**.

A convex function finite everywhere is lower semicontinuous; in finite dimensions, a lower semicontinuous **coercive** function **attains its minimum** (e.g., any minimizing sequence is bounded by coercivity, then compactness and lower semicontinuity yield a limit minimizer). Hence $f$ attains a minimum.

Moreover, strong convexity implies **uniqueness**: if $X_1\neq X_2$ both minimized $f$, strong convexity would make the midpoint have strictly smaller value, a contradiction.

Therefore, on $\mathbb{R}^{p\times C}$, a strongly convex function **cannot** fail to attain its minimum.


We will now show that $f_\mu$ is smooth, i.e., $\nabla f_\mu$ is L-Lipschitz with respect to the Frobenius norm, with a simple conservative bound
$$
 L = \|\mathbf{A}\|_F^2 + \mu.
$$
where
$$
 \mathbf{A} = \begin{bmatrix}
  \leftarrow &  \mathbf{a}_1^\top & \rightarrow \\
  \leftarrow &  \mathbf{a}_2^\top & \rightarrow \\
   &  \ldots &  \\
  \leftarrow &  \mathbf{a}_n^\top & \rightarrow \\
 \end{bmatrix}.
$$
(You may use that the operator norm of the softmax Jacobian is bounded by 1/4, and a looser bound $\le 1$ is acceptable for grading.)

Hint: check the properties of the spectral norm with respect to dot product, Kronecker product, and outer product.

(1 point for all three questions)


__(j-1)__ Show that $\lambda_{\max}(\mathbf{a}_i\mathbf{a}_i^T) = \left\| \mathbf{a}_i\right\|_2^2$, where $\lambda_{\max}(\cdot)$ denotes the largest eigenvalue.


### Solution

**Claim.** For any $\mathbf{a}_i\in\mathbb{R}^p$,
$$
\lambda_{\max}(\mathbf{a}_i\mathbf{a}_i^\top)=\|\mathbf{a}_i\|_2^2.
$$

**Proof.** The matrix $\mathbf{a}_i\mathbf{a}_i^\top$ is symmetric, positive semidefinite, and rank one.

1) **Exhibit an eigenpair.**  
   Compute
   $$
   (\mathbf{a}_i\mathbf{a}_i^\top)\mathbf{a}_i
   = \mathbf{a}_i(\mathbf{a}_i^\top\mathbf{a}_i)
   = \|\mathbf{a}_i\|_2^2\,\mathbf{a}_i,
   $$
   so $\mathbf{a}_i$ is an eigenvector with eigenvalue $\|\mathbf{a}_i\|_2^2$.

2) **Upper bound any eigenvalue by the Rayleigh quotient.**  
   For any unit vector $\mathbf{x}$,
   $$
   \mathbf{x}^\top(\mathbf{a}_i\mathbf{a}_i^\top)\mathbf{x}
   = (\mathbf{a}_i^\top\mathbf{x})^2
   \le \|\mathbf{a}_i\|_2^2\|\mathbf{x}\|_2^2
   = \|\mathbf{a}_i\|_2^2,
   $$
   where the inequality is Cauchy-Schwarz. Hence $\lambda_{\max}\le \|\mathbf{a}_i\|_2^2$.

Combining with step 1 (which shows $\lambda_{\max}\ge \|\mathbf{a}_i\|_2^2$), we obtain equality:
$$
\boxed{\ \lambda_{\max}(\mathbf{a}_i\mathbf{a}_i^\top)=\|\mathbf{a}_i\|_2^2\ }.
$$

(Every other eigenvalue equals $0$ because $\mathbf{a}_i\mathbf{a}_i^\top$ is rank one.)


__(j-2)__ Using (2), show that $\lambda_{\max}(\nabla^2 f_\mu(\mathbf{X})) \leq \sum_{i=1}^{n} \|\mathbf{a}_i\|_2^2 + \mu$.

### Solution

From (g),
$$
\nabla^2 f_\mu(\mathbf{X})
=\sum_{i=1}^n (\mathbf{a}_i\mathbf{a}_i^\top)\otimes J_i + \mu I,
\qquad
J_i=\mathrm{Diag}(\mathbf{p}_i)-\mathbf{p}_i\mathbf{p}_i^\top \succeq 0 .
$$

For PSD matrices, $\lambda_{\max}(M)=\|M\|_2$. Using standard spectral-norm properties:

1. **Sum:** $\| \sum_i M_i \|_2 \le \sum_i \|M_i\|_2$.
2. **Kronecker:** $\|A\otimes B\|_2=\|A\|_2\,\|B\|_2$.
3. **Rank-one:** from (j-1), $\|\mathbf{a}_i\mathbf{a}_i^\top\|_2=\lambda_{\max}(\mathbf{a}_i\mathbf{a}_i^\top)=\|\mathbf{a}_i\|_2^2$.
4. **Softmax Jacobian:** $\|J_i\|_2 \le 1$ (tighter bound $\le\frac14$ also holds).

Hence, for each $i$,
$$
\|(\mathbf{a}_i\mathbf{a}_i^\top)\otimes J_i\|_2
=\|\mathbf{a}_i\mathbf{a}_i^\top\|_2\,\|J_i\|_2
\le \|\mathbf{a}_i\|_2^2.
$$
Therefore,
$$
\lambda_{\max}\!\big(\nabla^2 f_\mu(\mathbf{X})\big)
= \Big\|\sum_{i=1}^n (\mathbf{a}_i\mathbf{a}_i^\top)\otimes J_i + \mu I \Big\|_2
\le \sum_{i=1}^n \|(\mathbf{a}_i\mathbf{a}_i^\top)\otimes J_i\|_2 + \|\mu I\|_2
\le \sum_{i=1}^n \|\mathbf{a}_i\|_2^2 + \mu.
$$

*(If you use the tighter bound $\|J_i\|_2 \le \tfrac14$, you obtain
$\lambda_{\max}(\nabla^2 f_\mu(\mathbf{X})) \le \tfrac14 \sum_i \|\mathbf{a}_i\|_2^2 + \mu$.)*


__(j-3)__ Conclude that $f_\mu$ is $L$-smooth for $L = \|\mathbf{A}\|_F^2 + \mu$.


### Solution

From (j-2), for every $\mathbf{X}$,
$$
\lambda_{\max}\!\big(\nabla^2 f_\mu(\mathbf{X})\big)\;\le\;\sum_{i=1}^n \|\mathbf{a}_i\|_2^2+\mu .
$$
Let $\mathbf{A}\in\mathbb{R}^{n\times p}$ have rows $\mathbf{a}_i^\top$. Then
$$
\sum_{i=1}^n \|\mathbf{a}_i\|_2^2 \;=\; \|\mathbf{A}\|_F^2 ,
$$
so
$$
\sup_{\mathbf{X}} \lambda_{\max}\!\big(\nabla^2 f_\mu(\mathbf{X})\big) \;\le\; \|\mathbf{A}\|_F^2+\mu .
$$

For a twice-differentiable function, the Lipschitz constant of the gradient (with respect to $\|\cdot\|_F$) can be taken as any upper bound on $\sup_{\mathbf{X}}\lambda_{\max}(\nabla^2 f_\mu(\mathbf{X}))$. Hence, for all $\mathbf{X},\mathbf{Y}$,
$$
\|\nabla f_\mu(\mathbf{X})-\nabla f_\mu(\mathbf{Y})\|_F
\;\le\;
\big(\sup_{\mathbf{Z}}\lambda_{\max}(\nabla^2 f_\mu(\mathbf{Z}))\big)\,\|\mathbf{X}-\mathbf{Y}\|_F
\;\le\; (\|\mathbf{A}\|_F^2+\mu)\,\|\mathbf{X}-\mathbf{Y}\|_F .
$$

Therefore, $f_\mu$ is $L$-smooth with
$$
\boxed{\,L=\|\mathbf{A}\|_F^2+\mu\, }.
$$


__(l)__ (1 point) KL divergence and NLL. Let $q(b_i\mid\mathbf{a}_i)$ be the true label distribution and $p(b_i\mid\mathbf{a}_i)$ the model softmax. Write the KL divergence $\mathrm{KL}(q\,\|\,p)$ and show that minimizing the KL divergence between $q$ and $p$ is equivalent to minimizing the negative log-likelihood derived in (a).


### Solution

Let $p_i(c)=\mathbb{P}(b_i=c\mid \mathbf{a}_i;\mathbf{X})$ be the model softmax for example $i$ and
$q_i(c)=q(b_i=c\mid \mathbf{a}_i)$ the (unknown/true) label distribution.

The KL divergence between $q$ and $p$ over the dataset is
$$
\sum_{i=1}^n \mathrm{KL}(q_i\|p_i)
=\sum_{i=1}^n \sum_{c=1}^C q_i(c)\,\log\frac{q_i(c)}{p_i(c)}
= -\underbrace{\sum_{i=1}^n \sum_{c=1}^C q_i(c)\log p_i(c)}_{\text{cross-entropy }H(q,p)}
\;+\; \underbrace{\sum_{i=1}^n \sum_{c=1}^C q_i(c)\log q_i(c)}_{-\sum_i H(q_i)} .
$$
The second term depends only on $q$ (not on $\mathbf{X}$), hence it is a constant for optimization over $\mathbf{X}$.
Therefore,
$$
\arg\min_{\mathbf{X}} \sum_{i=1}^n \mathrm{KL}(q_i\|p_i)
\;=\; \arg\min_{\mathbf{X}} H(q,p)
\;=\; \arg\max_{\mathbf{X}} \sum_{i=1}^n \sum_{c=1}^C q_i(c)\log p_i(c).
$$

**Hard labels (one-hot).**  
If each training example has a single observed label $b_i$, then
$q_i(c)=\mathbf{1}\{c=b_i\}=\mathbf{e}_{b_i,c}$, and the cross-entropy becomes
$$
H(q,p) \;=\; -\sum_{i=1}^n \log p_i(b_i).
$$
Under the softmax model,
$$
-\sum_{i=1}^n \log p_i(b_i)
= \sum_{i=1}^n \left[\log\!\left(\sum_{k=1}^C e^{\mathbf{a}_i^\top \mathbf{x}_k}\right)
- \mathbf{a}_i^\top \mathbf{x}_{b_i}\right],
$$
which is exactly the negative log-likelihood from part (a).

**Conclusion.** Minimizing $\sum_i \mathrm{KL}(q_i\|p_i)$ is equivalent to minimizing the NLL (cross-entropy) used in multiclass logistic regression; for one-hot $q$, the KL equals the NLL up to the additive constant $\sum_i H(q_i)$ (which is $0$ for one-hot labels).


From your work in this section, you have shown that the maximum likelihood estimator for multiclass softmax logistic regression might not exist, but it can be guaranteed to exist by adding a $\|\cdot\|_F^2$ regularizer. Consequently, the estimator for $\mathbf{X}$ we will use will be the solution of the smooth strongly convex problem,
$$
 \mathbf{X}^\star = \arg\min_{\mathbf{X} \in \mathbb{R}^{p\times C}} f(\mathbf{X}) + \frac{\mu}{2}\|\mathbf{X}\|_F^2.\tag{3}
$$


## Binary logistic regression (specialization for Part 2)

While this part analyzed the multiclass (softmax) setting, in the next exercise we will continue under the simplified two-class case.

Let labels be $b_i \in \{-1, +1\}$, features $\mathbf{a}_i \in \mathbb{R}^p$, and weight vector $\mathbf{x} \in \mathbb{R}^p$. Define the sigmoid
$$
\sigma(t) = \frac{1}{1+e^{-t}}.
$$
Model the conditional distribution as
$$
\mathbb{P}(b_i = j \mid \mathbf{a}_i) = \sigma\big(j\, \mathbf{a}_i^\top \mathbf{x}\big), \quad j \in \{-1,+1\}.
$$
The likelihood over i.i.d. samples $\{(\mathbf{a}_i, b_i)\}_{i=1}^n$ is
$$
\mathcal{L}(\mathbf{x}) = \prod_{i=1}^n \sigma\big(b_i\, \mathbf{a}_i^\top \mathbf{x}\big),
$$
so the negative log-likelihood is
$$
 f(\mathbf{x}) = -\log \mathcal{L}(\mathbf{x}) = \sum_{i=1}^n \log\big(1 + e^{-b_i\, \mathbf{a}_i^\top \mathbf{x}}\big).
$$

__(m)__ (1 point) Show that the gradient of the negative log-likelihood is the standard binary logistic regression gradient:
$$
\nabla f(\mathbf{x}) = \sum_{i=1}^n \big(-b_i\, \sigma(-b_i\, \mathbf{a}_i^\top \mathbf{x})\big)\, \mathbf{a}_i.
$$
(Hint: use the chain rule and $\sigma'(t) = \sigma(t)\big(1-\sigma(t)\big)$.)

We will use this binary formulation in Part 2 - First order methods.


### Solution

Let labels $b_i\in\{-1,+1\}$, features $\mathbf{a}_i\in\mathbb{R}^p$, weights $\mathbf{x}\in\mathbb{R}^p$, and
$$
f(\mathbf{x}) \;=\; \sum_{i=1}^n \log\!\big(1+e^{-\,b_i\,\mathbf{a}_i^\top \mathbf{x}}\big).
$$

For each term define
$$
t_i(\mathbf{x}) := -\,b_i\,\mathbf{a}_i^\top \mathbf{x},
\qquad
g(t) := \log(1+e^{t}),
$$
so $f(\mathbf{x})=\sum_i g\!\big(t_i(\mathbf{x})\big)$.

- $g'(t)=\dfrac{e^{t}}{1+e^{t}}=\sigma(t)$, where $\sigma(t)=\dfrac{1}{1+e^{-t}}$.
- $\nabla_{\mathbf{x}} t_i(\mathbf{x}) = -\,b_i\,\mathbf{a}_i$.

By the chain rule,
$$
\nabla f(\mathbf{x})
= \sum_{i=1}^n g'\!\big(t_i(\mathbf{x})\big)\,\nabla t_i(\mathbf{x})
= \sum_{i=1}^n \sigma\!\big(t_i(\mathbf{x})\big)\,(-\,b_i\,\mathbf{a}_i)
= \sum_{i=1}^n \big[-\,b_i\,\sigma(-\,b_i\,\mathbf{a}_i^\top \mathbf{x})\big]\mathbf{a}_i.
$$

This is the standard binary logistic-regression gradient.

**Equivalent 0/1-label form.**  
If $y_i=\dfrac{b_i+1}{2}\in\{0,1\}$, then using $\sigma(-b_i z)=\sigma(z)-y_i$ we get
$$
\nabla f(\mathbf{x}) \;=\; \sum_{i=1}^n \big(\sigma(\mathbf{a}_i^\top \mathbf{x})-y_i\big)\,\mathbf{a}_i.
$$
